In [5]:
##############################################
# Import Library
##############################################

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split

from imblearn.datasets import fetch_datasets

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy.stats import bernoulli, zscore

import calibration_metric
import importlib
importlib.reload(calibration_metric)
from calibration_metric import ece, ace, tce

In [6]:
##############################################
# Prepare Algorithms
##############################################

def pick_algorithm(name_alg):
    if name_alg == "LR":
        return LogisticRegression(max_iter=10000, random_state=0)
    elif name_alg == "SVM":
        return SVC(kernel='rbf', probability=True, random_state=0)
    elif name_alg == "RF":
        return RandomForestClassifier(max_depth=10, random_state=0)
    elif name_alg == "GB":
        return GradientBoostingClassifier(random_state=0)
    elif name_alg == "MLP":
        return MLPClassifier(hidden_layer_sizes=(1000), max_iter=10000, random_state=0)
    else:
        assert False, "No Algorithm"

names_alg = ['LR', 'SVM', 'RF', 'GB', 'MLP']


In [7]:
##############################################
# Prepare Data
##############################################

# Utility Function
def process(df, test_size=0.3):
    df_X = df.data
    df_y = df.target.astype(int)
    m = df_X.mean(axis=0)
    s = df_X.std(axis=0)
    s[s==0] = 1
    df_X = ( df_X - m ) / s
    df_y[df_y == -1] = 0
    df_train_X, df_test_X = train_test_split(df_X, test_size=test_size, random_state=0)
    df_train_y, df_test_y = train_test_split(df_y, test_size=test_size, random_state=0)
    return (df_train_X, df_train_y, df_test_X, df_test_y)


def process_11(df, y_column, pos_index=1, test_size=0.3):
    if isinstance(y_column, str):
        df_X = pd.get_dummies(df.loc[:, df.columns != y_column])
        df_y = pd.get_dummies(df.loc[:, y_column]).to_numpy()[:, pos_index]
    else:
        df_X = pd.get_dummies(df.iloc[:, list(set(range(df.shape[1])) - set([y_column]))])
        df_y = pd.get_dummies(df.iloc[:, y_column]).to_numpy()[:, pos_index]
    
    df_X.loc[:, df_X.dtypes=='float'] = ( df_X.loc[:, df_X.dtypes=='float'] - df_X.loc[:, df_X.dtypes=='float'].mean() ) / df_X.loc[:, df_X.dtypes=='float'].std()
    df_X = df_X.to_numpy()
    
    df_train_X, df_test_X = train_test_split(df_X, test_size=test_size, random_state=0)
    df_train_y, df_test_y = train_test_split(df_y, test_size=test_size, random_state=0)

    return (df_train_X, df_train_y, df_test_X, df_test_y)


# Data 11
df11 = pd.read_csv('./Dataset/UCI/spambase.data', header=None, dtype='float')
df11.iloc[:,57] = df11.iloc[:,57].astype('category')


# Data All
names_df = [ 'spambase', 'optical_digits', 'satimage', 'pen_digits', 'abalone', 'isolet', 'coil_2000', 'letter_img', 'webpage', 'mammography' ]
dfs = [ process_11(df11, 57) ] + [ process(fetch_datasets()[name_df]) for name_df in names_df[1:] ]


# Check Number
for ith, ( df_train_X, df_train_y, df_test_X, df_test_y ) in enumerate(dfs):
    print(str(ith+1) + " - " + str(len(df_test_y)) + " - " + str(sum(df_test_y==1)))


1 - 1381 - 559
2 - 1686 - 151
3 - 1931 - 207
4 - 3298 - 303
5 - 1254 - 132
6 - 2340 - 170
7 - 2947 - 171
8 - 6000 - 211
9 - 10434 - 292
10 - 3355 - 77


In [8]:
##############################################
# Fit Algrithms and Measure Errors
##############################################

def do_experiment(dfs, names_df, names_alg):
    tces = np.zeros((len(names_df), len(names_alg)))
    taes = np.zeros((len(names_df), len(names_alg)))
    eces = np.zeros((len(names_df), len(names_alg)))
    aces = np.zeros((len(names_df), len(names_alg)))
    mces = np.zeros((len(names_df), len(names_alg)))
    maes = np.zeros((len(names_df), len(names_alg)))
    
    for ith, (df_train_X, df_train_y, df_test_X, df_test_y) in enumerate(dfs):
        print("- Data: " + names_df[ith])
        for jth in range(len(names_alg)):
            print("- - Processing: " + names_alg[jth])
            
            alg = pick_algorithm(names_alg[jth])
            alg.fit(df_train_X, df_train_y)
            preds_test = alg.predict_proba(df_test_X)[:,1]
            
            filename = "./Plot/UCI/" + names_df[ith] + "_" + names_alg[jth]
            n_min = int( len(df_test_y) / 20 )
            n_max = int( len(df_test_y) / 5 )
            n_bin = 10
            
            tces[ith, jth] = tce(preds_test, df_test_y, strategy="pavabc", n_min=n_min, n_max=n_max, savepath=filename+"_tce_pavabc.png")
            taes[ith, jth] = tce(preds_test, df_test_y, strategy="quantile", n_bin=n_bin, savepath=filename+"_tce_quantile.png")
            eces[ith, jth] = ece(preds_test, df_test_y, n_bin=n_bin, mode='l1', savepath=filename+"_ece.png")
            aces[ith, jth] = ace(preds_test, df_test_y, n_bin=n_bin, mode='l1', savepath=filename+"_ace.png")
            mces[ith, jth] = ece(preds_test, df_test_y, n_bin=n_bin, mode='inf')
            maes[ith, jth] = ace(preds_test, df_test_y, n_bin=n_bin, mode='inf')
        
        print("- Completed")
        filename = "./Result/UCI/" + names_df[ith] + ".csv"
        result = pd.DataFrame(np.c_[ tces[ith, :], taes[ith, :], eces[ith, :], aces[ith, :], mces[ith, :], maes[ith, :] ])
        result.index = names_alg
        result.columns = ['TCE(P)', 'TCE(Q)', 'ECE', 'ACE', 'MCE', 'MCE(Q)']
        result.to_csv(filename, float_format="%.4f")
        pd.options.display.float_format = '{:.4f}'.format
        display(result)

        
do_experiment(dfs, names_df, names_alg)




- Data: spambase
- - Processing: LR
- - Processing: SVM
- - Processing: RF
- - Processing: GB
- - Processing: MLP
- Completed


,TCE(P),TCE(Q),ECE,ACE,MCE,MCE(Q)
LR,33.6713,56.1188,0.0256,0.0267,0.1539,0.0895
SVM,12.8168,34.5402,0.0177,0.0227,0.2207,0.0465
RF,66.0391,49.4569,0.0635,0.0601,0.2056,0.1616
GB,20.2028,20.4200,0.0295,0.0277,0.1409,0.0891
MLP,60.9703,67.1253,0.0413,0.0397,0.2931,0.1076


- Data: optical_digits
- - Processing: LR
- - Processing: SVM
- - Processing: RF
- - Processing: GB
- - Processing: MLP
- Completed


,TCE(P),TCE(Q),ECE,ACE,MCE,MCE(Q)
LR,11.6251,27.1649,0.0098,0.0037,0.2251,0.0135
SVM,4.8043,10.6762,0.0042,0.0028,0.6608,0.0157
RF,49.6441,38.3155,0.0451,0.0433,0.5432,0.2271
GB,13.0486,11.2693,0.0181,0.0168,0.5639,0.1122
MLP,4.6856,12.1590,0.0037,0.0034,0.5992,0.0306


- Data: satimage
- - Processing: LR
- - Processing: SVM
- - Processing: RF
- - Processing: GB
- - Processing: MLP
- Completed


...

,TCE(P),TCE(Q),ECE,ACE,MCE,MCE(Q)
LR,10.5167,12.0500,0.0025,0.0008,0.1617,0.0042
SVM,11.8667,14.8167,0.0019,0.0017,0.6257,0.0146
RF,26.5000,20.2500,0.0097,0.0033,0.5179,0.0131
GB,25.9500,18.7333,0.0067,0.0029,0.3653,0.0109
MLP,19.9833,9.9833,0.0010,0.0001,0.4550,0.0007


- Data: webpage
- - Processing: LR
- - Processing: SVM
- - Processing: RF
- - Processing: GB
- - Processing: MLP
- Completed


,TCE(P),TCE(Q),ECE,ACE,MCE,MCE(Q)
LR,40.1572,44.4604,0.0044,0.0034,0.3134,0.0214
SVM,59.8332,39.7738,0.0043,0.0057,0.5402,0.0239
RF,99.6550,99.6550,0.0234,0.0241,0.5980,0.1189
GB,71.1233,79.8543,0.0086,0.0107,0.2399,0.0436
MLP,49.8083,39.9367,0.0090,0.0018,0.4344,0.0076


- Data: mammography
- - Processing: LR
- - Processing: SVM
- - Processing: RF
- - Processing: GB
- - Processing: MLP
- Completed


,TCE(P),TCE(Q),ECE,ACE,MCE,MCE(Q)
LR,25.0671,26.7660,0.0027,0.0065,0.3594,0.0208
SVM,20.2683,20.1490,0.0067,0.0088,0.6741,0.0353
RF,19.4039,9.2996,0.0047,0.0016,0.4465,0.0043
GB,14.5156,15.4098,0.0061,0.0034,0.5355,0.0124
MLP,20.5663,26.9747,0.0042,0.0027,0.4351,0.0113
